# NAMEX API Utils

In [ ]:
import os
from sqlalchemy import desc
import datedelta
import base64
import uuid

In [ ]:
NAMEX_AUTH_SVC_URL = os.getenv('NAMEX_AUTH_SVC_URL')
NAMEX_SERVICE_CLIENT_USERNAME = os.getenv('NAMEX_SERVICE_CLIENT_USERNAME')
NAMEX_SERVICE_CLIENT_SECRET = os.getenv('NAMEX_SERVICE_CLIENT_SECRET')
NAMEX_SVC_URL = os.getenv('NAMEX_SVC_URL')
PAYMENT_SVC_URL = os.getenv('PAYMENT_SVC_URL')
PAYMENT_SVC_CLIENT_SYSTEM_NAMEX_URL = os.getenv('PAYMENT_SVC_CLIENT_SYSTEM_NAMEX_URL')
PAYMENT_SVC_RETURN_URL = os.getenv('PAYMENT_SVC_RETURN_URL')

In [ ]:
def get_namex_token(verify_ssl = True):
    payload = "grant_type=client_credentials"
    basic_hash = base64.b64encode(f'{NAMEX_SERVICE_CLIENT_USERNAME}:{NAMEX_SERVICE_CLIENT_SECRET}'.encode()).decode()
    basic_hash
    headers = {
      'Content-Type': 'application/x-www-form-urlencoded',
      'Authorization': f'Basic {basic_hash}'
    }

    response = requests.request("POST", NAMEX_AUTH_SVC_URL, headers=headers, data=payload, verify=verify_ssl)

    namex_token = response.json()['access_token']
    # namex_token
    assert namex_token
    return namex_token    

In [ ]:
def get_nr_approved_name(nr_num: str, email, verify_ssl = True):
    url = f'{NAMEX_SVC_URL}/namerequests'

    headers = {
      'BCREG-User-Email': email,
      'BCREG-NR': nr_num,
      'BCREG-NRL': '',
      'BCREG-User-Phone': ''
    }

    r = requests.get(url, headers=headers, verify=verify_ssl)
    r.status_code, r.text

    response_dict = json.loads(r.text)
    names = response_dict['names']
    matching_approved_names = list(filter(lambda x: x['state'] == 'APPROVED', names))
    # print(f'matching_approved_names: {matching_approved_names}, matches: {len(matching_approved_names)}')
    assert len(matching_approved_names) == 1
    approved_nr_name = matching_approved_names[0]['name']
    approved_nr_name
    return approved_nr_name

In [ ]:
def create_nr(payload: dict, verify_ssl = True):
    url = f'{NAMEX_SVC_URL}/namerequests'
    
    headers = {
      'Content-Type': 'application/json',
    }

    payload = json.dumps(create_nr_json, indent=2)

    r = requests.request("POST", url, headers=headers, data=payload, verify=verify_ssl)
    r.status_code, r.text
    return r

In [ ]:
def create_nr_payment(nr_id: int, nrl: str, email: str, payload: dict, verify_ssl = True):
    url = f'{NAMEX_SVC_URL}/payments/{nr_id}/CREATE'
    print(f'create payment url: {url}')


    headers = {
      'Content-Type': 'application/json',
      'BCREG-User-Email': email,
      'BCREG-NRL': nrl
    }

    payload = json.dumps(create_payment_json, indent=2)

    r = requests.request("POST", url, headers=headers, data=payload, verify=verify_ssl)
    return r

In [ ]:
def create_nr_payment_transaction(nr_id: int, payment_id: int, payment_token: str, verify_ssl = True):
    url = f'{PAYMENT_SVC_URL}/{payment_token}/transactions'
    payment_svc_client_sys_url = PAYMENT_SVC_CLIENT_SYSTEM_NAMEX_URL.format(nr_id = nr_id, payment_id = payment_id)

    create_transaction_json = {
      'clientSystemUrl': payment_svc_client_sys_url,
      'payReturnUrl': PAYMENT_SVC_RETURN_URL
    }
    
    headers = {
      'Content-Type': 'application/json'
    }
    payload = json.dumps(create_transaction_json, indent=2)

    r = requests.request("POST", url, headers=headers, data=payload, verify=verify_ssl)
    return r

In [ ]:
def update_nr_state(namex_token: str, nr_num: str, payload: dict, verify_ssl = True):
    url = f'{NAMEX_SVC_URL}/requests/{nr_num}'
    
    headers = {
      'Content-Type': 'application/json',
      'Authorization': f'Bearer {namex_token}'
    }

    payload = json.dumps(payload, indent=2)

    r = requests.request("PATCH", url, headers=headers, data=payload, verify=verify_ssl)
    return r

In [ ]:
def update_nr_name(namex_token: str, nr_num: str, payload: dict, verify_ssl = True):
    url = f'{NAMEX_SVC_URL}/requests/{nr_num}/names/1'
    
    headers = {
      'Content-Type': 'application/json',
      'Authorization': f'Bearer {namex_token}'
    }

    payload = json.dumps(payload, indent=2)

    r = requests.request("PUT", url, headers=headers, data=payload, verify=verify_ssl)
    return r